In [1]:
with open("../data/day15.txt") as f:
    data = f.read().split("\n\n")

In [2]:
import numpy as np

In [3]:
arr = np.array([list(row) for row in data[0].split("\n")])
moves = [move for move in data[1] if move in "^v<>"]

In [4]:
def get_relevant_row(arr, move):
    robotposition = np.argwhere(arr == "@")[0]
    if move == "<":
        return arr[robotposition[0], : robotposition[1] + 1][::-1]
    if move == ">":
        return arr[robotposition[0], robotposition[1] :]
    if move == "^":
        return arr[: robotposition[0] + 1, robotposition[1]][::-1]
    if move == "v":
        return arr[robotposition[0] :, robotposition[1]]


def length_moving(row):
    i = 0
    for object in row:
        if object in {"@", "O"}:
            i += 1
        elif object == ".":
            return i
        elif object == "#":
            # Nothing can move
            return 0


def moving_row(length_moving):
    return ["@"] + ["O"] * (length_moving - 1)

In [ ]:
temp_arr = arr.copy()
for move in moves:
    relevant_row = get_relevant_row(temp_arr, move)
    moving = length_moving(relevant_row)
    if moving == 0:
        continue
    robotposition = np.argwhere(temp_arr == "@")[0]
    if move == ">":
        temp_arr[
            robotposition[0], robotposition[1] : robotposition[1] + (moving + 1)
        ] = ["."] + moving_row(moving)
    if move == "<":
        temp_arr[robotposition[0], robotposition[1] - moving : robotposition[1] + 1] = (
            ["."] + moving_row(moving)
        )[::-1]
    if move == "^":
        temp_arr[robotposition[0] - moving : robotposition[0] + 1, robotposition[1]] = (
            ["."] + moving_row(moving)
        )[::-1]
    if move == "v":
        temp_arr[
            robotposition[0] : robotposition[0] + (moving + 1), robotposition[1]
        ] = ["."] + moving_row(moving)

temp_arr

In [ ]:
def calculate_gps(arr):
    goods_positions = np.argwhere(arr == "O")
    score = 0
    for position in goods_positions:
        score += position[0] * 100 + position[1]
    return score


calculate_gps(temp_arr)

# Part 2

In [ ]:
def scale_up_map(arr):
    total = []
    for y in range(arr.shape[0]):
        row = []
        for x in range(arr.shape[1]):
            scaled_up = {"#": "##", "O": "[]", ".": "..", "@": "@."}
            row.extend(scaled_up[arr[y, x]])
        total.append(row)
    return np.array(total)


def get_relevant_row(arr, move):
    robotposition = np.argwhere(arr == "@")[0]
    if move == "<":
        return arr[robotposition[0], : robotposition[1] + 1][::-1]
    if move == ">":
        return arr[robotposition[0], robotposition[1] :]
    if move == "^":
        return arr[: robotposition[0] + 1, robotposition[1]][::-1]
    if move == "v":
        return arr[robotposition[0] :, robotposition[1]]


def add_following_row(arr, current_indices, direction):
    direction = -1 if direction == "^" else 1
    next_indices = []
    for y, x in current_indices:
        if arr[y + direction, x] == "[":
            next_indices.extend([(y + direction, x), (y + direction, x + 1)])
        elif arr[y + direction, x] == "]":
            next_indices.extend([(y + direction, x - 1), (y + direction, x)])
        elif arr[y + direction, x] == "#":
            raise ValueError("Can not move")
    return next_indices


def return_moving_indices_vertical(arr, direction):
    robotposition = np.argwhere(arr == "@")[0]
    moving_indices = [tuple(robotposition)]
    next_indices = [tuple(robotposition)]
    while next_indices:
        try:
            next_indices = add_following_row(arr, next_indices, direction)
        except ValueError:
            return []
        moving_indices.extend(next_indices)
    return moving_indices


def return_moving_indices_horizontal(arr, direction):
    robotposition = np.argwhere(arr == "@")[0]
    direction = -1 if direction == "<" else 1
    moving_indices = []
    current_object_index = robotposition
    while True:
        field = arr[tuple(current_object_index)]
        if field in "@[]":
            moving_indices.append(tuple(current_object_index))
            current_object_index += np.array([0, 1]) * direction
        elif field == ".":
            return moving_indices
        elif field == "#":
            return []


def return_moving_indices(arr, direction):
    if direction in "^v":
        moving_indices = return_moving_indices_vertical(arr, direction)
    if direction in "<>":
        moving_indices = return_moving_indices_horizontal(arr, direction)

    # Make unique without changing order
    return list(dict.fromkeys(moving_indices))


def move_index(arr, index, move):
    if move == "^":
        arr[index[0] - 1, index[1]] = arr[index[0], index[1]]
    if move == "v":
        arr[index[0] + 1, index[1]] = arr[index[0], index[1]]
    if move == ">":
        arr[index[0], index[1] + 1] = arr[index[0], index[1]]
    if move == "<":
        arr[index[0], index[1] - 1] = arr[index[0], index[1]]
    arr[index[0], index[1]] = "."
    return arr


arr_scaled = scale_up_map(arr)

temp_arr = arr_scaled.copy()
for move in moves:
    moving_indices = return_moving_indices(temp_arr, move)
    for moving_index in moving_indices[::-1]:
        temp_arr = move_index(temp_arr, moving_index, move)

print("\n".join("".join(row) for row in temp_arr))

In [ ]:
def scaled_up_gps(arr):
    goods_positions = np.argwhere(arr == "[")
    score = 0
    for position in goods_positions:
        score += position[0] * 100 + position[1]
    return score


scaled_up_gps(temp_arr)